In [1]:
# in case there are any problems with importing because path is wrong
import sys
sys.path.append('/Users/daniel/Princeton Dropbox/Daniel Gurevich/Research/discrete_sr/code/SPIDER_discrete')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import rcParams
import h5py
import matplotlib.pyplot as plt

from library import Observable
from continuous.process_library_terms import SRDataset
from commons.utils import save, load

In [3]:
hdf = True
if hdf:
    import h5py
    
    def load_matlab_v73(mat_file_path):
        """
        Loads MATLAB v7.3 .mat data using h5py and converts it to Python-readable formats.
    
        Parameters:
        - mat_file_path (str): Path to the .mat file.
    
        Returns:
        - dict: A dictionary with MATLAB variable names as keys and corresponding NumPy arrays as values.
        """
        try:
            # Open the HDF5 file
            with h5py.File(mat_file_path, 'r') as f:
                mat_data = {}
    
                def recursively_load(group):
                    """
                    Recursively load MATLAB v7.3 groups into dictionaries.
                    """
                    data = {}
                    for key, item in group.items():
                        if isinstance(item, h5py.Dataset):
                            data[key] = np.array(item)  # Convert HDF5 dataset to NumPy array
                        elif isinstance(item, h5py.Group):
                            data[key] = recursively_load(item)  # Recursively process groups
                    return data
    
                # Load all variables from the root group
                mat_data = recursively_load(f)
    
            return mat_data
        except Exception as e:
            print(f"Error loading .mat file: {e}")
            return None
    
    # Path to your MATLAB v7.3 .mat file
    mat_file_path = "data/filcoh_SGS.mat" # Replace with your .mat file path

In [4]:
if hdf:
    # Load the .mat file
    python_data = load_matlab_v73(mat_file_path)
    
    # Display the loaded data
    if python_data:
        for var_name, data in python_data.items():
            if isinstance(data, np.ndarray):
                print(f"Variable: {var_name}, Shape: {data.shape}, Type: {type(data)}")
            else:
                print(f"Variable: {var_name}, Type: {type(data)} (nested structure)")
    
    s = python_data['s']  # Replace 's' with the actual key name if it's different
    
    # Extract the first layer (V) and the second layer (U)
    V = s[:, 0, :, :]  # First layer
    U = s[:, 1, :, :]  # Second layer
    
    # Transpose to correct shape
    
    U = np.transpose(U, (2,1,0))
    V = np.transpose(V, (2,1,0))

    print(U.shape, V.shape)

    #SUBSAMPLE
    xsample = 2
    ysample = xsample
    tsample = 1
    
    U = U[::xsample, ::ysample, ::tsample]
    V = V[::xsample, ::ysample, ::tsample]
        
    Lx = 2*np.pi; Ly = 2*np.pi; Lt = 5;
    Nx = 2048/xsample; Ny = 2048/ysample; Nt = 100/tsample
    dx = Lx/Nx; dy = Ly/Ny; dt = Lt/Nt;
    
    def pressure_poisson(U, V, dx, dy, density=1.0):
        nx, ny, nt = U.shape
        kx = np.fft.fftfreq(nx, d=dx) * 2 * np.pi
        ky = np.fft.rfftfreq(ny, d=dy) * 2 * np.pi
        kx, ky = np.meshgrid(kx, ky, indexing='ij')
        k_squared = kx**2 + ky**2
        k_squared[0, 0] = np.inf 
        P = np.zeros((nx, ny, nt))
        for t in range(nt):
            u_FT = np.fft.rfftn(U[:, :, t])
            v_FT = np.fft.rfftn(V[:, :, t])
            i = 1j
            dxu = np.fft.irfftn(i * kx * u_FT, s=(nx, ny))
            dyu = np.fft.irfftn(i * ky * u_FT, s=(nx, ny))
            dxv = np.fft.irfftn(i * kx * v_FT, s=(nx, ny))
            dyv = np.fft.irfftn(i * ky * v_FT, s=(nx, ny))
            rhs = dxu**2 + 2 * dyu * dxv + dyv**2
            rhs_FT = np.fft.rfftn(rhs)
            pressure_FT = density * rhs_FT / k_squared
            p = np.fft.irfftn(pressure_FT, s=(nx, ny))
            P[:, :, t] = p
        return P
    
    P = pressure_poisson(U, V, dx, dy)
    
    print(f"V: Shape = {V.shape}, Type = {type(V)}")
    print(f"U: Shape = {U.shape}, Type = {type(U)}")
    print(f"P: Shape = {P.shape}, Type = {type(P)}")

    u = np.concatenate([U[:, :, :, np.newaxis], V[:, :, :, np.newaxis]], axis=3)

Variable: #refs#, Type: <class 'dict'> (nested structure)
Variable: #subsystem#, Type: <class 'dict'> (nested structure)
Variable: domain, Type: <class 'dict'> (nested structure)
Variable: params, Type: <class 'dict'> (nested structure)
Variable: s, Shape: (100, 2, 2048, 2048), Type: <class 'numpy.ndarray'>
(2048, 2048, 100) (2048, 2048, 100)


/var/folders/p_/dc9h9qcj62qgv461rz72zx_80000gn/T/ipykernel_52174/3612960488.py:50: DeprecationWarning: `axes` should not be `None` if `s` is not `None` (Deprecated in NumPy 2.0). In a future version of NumPy, this will raise an error and `s[i]` will correspond to the size along the transformed axis specified by `axes[i]`. To retain current behaviour, pass a sequence [0, ..., k-1] to `axes` for an array of dimension k.
  dxu = np.fft.irfftn(i * kx * u_FT, s=(nx, ny))
/var/folders/p_/dc9h9qcj62qgv461rz72zx_80000gn/T/ipykernel_52174/3612960488.py:51: DeprecationWarning: `axes` should not be `None` if `s` is not `None` (Deprecated in NumPy 2.0). In a future version of NumPy, this will raise an error and `s[i]` will correspond to the size along the transformed axis specified by `axes[i]`. To retain current behaviour, pass a sequence [0, ..., k-1] to `axes` for an array of dimension k.
  dyu = np.fft.irfftn(i * ky * u_FT, s=(nx, ny))
/var/folders/p_/dc9h9qcj62qgv461rz72zx_80000gn/T/ipykernel

V: Shape = (1024, 1024, 100), Type = <class 'numpy.ndarray'>
U: Shape = (1024, 1024, 100), Type = <class 'numpy.ndarray'>
P: Shape = (1024, 1024, 100), Type = <class 'numpy.ndarray'>


In [5]:
%%prun # profiling

uobs = Observable(string='u', rank=1)
pobs = Observable(string='p', rank=0)
observables = [uobs, pobs]
data_dict = {'p': P, 'u': u}

# fix random seed
np.random.seed(1)

world_size = np.array(P.shape)
pad = 0

# fix random seed
np.random.seed(1)

dxs = [dx, dy, dt]

# initial setup of dataset
srd = SRDataset(world_size=world_size, data_dict=data_dict, observables=observables, dxs=dxs, 
                irreps=SRDataset.all_rank2_irreps(), cache_primes=True)
                #irreps=SRDataset.only_rank2_irreps(), cache_primes=True)

# initialize libraries, domains, and weights
#srd.make_libraries(max_complexity=3, max_observables=3)
srd.make_libraries(max_complexity=4, max_observables=3)
for irrep in srd.irreps:
    print(irrep, ":", len(srd.libs[irrep].terms))

dom_width = 40
dom_time = 20 #previously 20 (without interpolation)
#srd.make_domains(ndomains=100, domain_size=[dom_width, dom_width, dom_time], pad=pad)
srd.make_domains(ndomains=30, domain_size=[dom_width, dom_width, dom_time], pad=pad)
#srd.make_domains(ndomains=10, domain_size=[dom_width, dom_width, dom_time], pad=pad)
srd.make_weights(m=12, qmax=0)
#srd.set_LT_scale(L=0.2, T=2.5e-3)
srd.set_LT_scale(L=0.2, T=0.5) #T=1 # note that this line must go before make_library_matrices
srd.make_library_matrices(debug=False)

Rank 0 : 41
Rank 1 : 49
Antisymmetric rank 2 : 23
Symmetric trace-free rank 2 : 36
 

         18577346 function calls (17747033 primitive calls) in 17.586 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    31410    2.925    0.000    5.264    0.000 process_library_terms.py:400(eval_term)
    94230    2.103    0.000    2.155    0.000 polynomial.py:672(polyval)
     9540    1.360    0.000    1.364    0.000 diff.py:479(_apply_to_array)
        2    0.926    0.463    1.357    0.679 _methods.py:144(_var)
       17    0.901    0.053    0.952    0.056 {built-in method time.sleep}
    94230    0.730    0.000    1.158    0.000 _function_base_impl.py:5078(trapezoid)
   233561    0.715    0.000    0.715    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        2    0.460    0.230    0.460    0.230 {method 'dot' of 'numpy.ndarray' objects}
    37620    0.441    0.000    0.538    0.000 numeric.py:978(tensordot)
    67980    0.413    0.000    1.118    0.000 arrayprint.py:966(fillFormat)
        4    0.372    0.093   14

In [6]:
# from commons.utils import *

# lib1 = srd.libs[srd.irreps[1]]
# for match in regex_find(lib1.terms, r'∂t u_α'):
#     print(match)
# dtu = lib1.Q[:, match[0]]

# for match in regex_find(lib1.terms, r'u_β · ∂β u_α'):
#     print(match)
# adv = lib1.Q[:, match[0]]

# for match in regex_find(lib1.terms, r'∂α p'):
#     print(match)
# dp = lib1.Q[:, match[0]]

# for match in regex_find(lib1.terms, r'∂β² u_α'):
#     print(match)
# viscosity = 0.02*lib1.Q[:, match[0]] #0.1

# print(np.linalg.norm(dtu), np.linalg.norm(adv), np.linalg.norm(dp), np.linalg.norm(dtu+dp+adv-viscosity), 
#       np.linalg.norm(dp-viscosity), np.linalg.norm(dtu+adv))

In [7]:
# lib0 = srd.libs[srd.irreps[0]]
# for match in regex_find(lib0.terms, r'∂α u_α'):
#     print(match)
# div = lib0.Q[:, match[0]]

# print(np.linalg.norm(div))

In [8]:
from commons.identify_models import *
import copy

libs = srd.libs

reg_opts_list = []
for irrep in srd.irreps:
    # for regression we now need to construct a Scaler, Initializer, ModelIterator, and Threshold
    scaler = Scaler(sub_inds=None, char_sizes=libs[irrep].col_weights, row_norms=None, unit_rows=True, train_fraction=1)
    init = Initializer(method='combinatorial', start_k=3)
    #init = Initializer(method='combinatorial', start_k=9999)
    #init = Initializer(method='power', start_k=10)
    #res = Residual(residual_type='fixed_column', anchor_col=0)
    res = Residual(residual_type='matrix_relative')
    
    iterator = ModelIterator(max_k=10, backward_forward=True, max_passes=2)
    #iterator = ModelIterator(max_k=len(libs[irrep].terms), backward_forward=False, max_passes=1)
    thres = Threshold(threshold_type='jump', gamma=1.5, delta=1e-10, n_terms=None)
    #thres = Threshold(threshold_type='information', ic=AIC)
    
    opts = {'scaler': scaler, 'initializer': init, 'residual': res,
            'model_iterator': iterator, 'threshold': thres}
    opts['verbose'] = False
    opts['inhomog'] = False
    opts['inhomog_col'] = None
    reg_opts_list.append(opts)

eqs, lambdas, reg_results, derived_eqs, excluded_terms = interleave_identify([libs[i] for i in srd.irreps], 
reg_opts_list, threshold=2e-6, experimental=True, report_accuracy=True)
#, max_equations=10)

--- WORKING ON LIBRARY WITH IRREP Rank 0 AT COMPLEXITY 1 ---
--- WORKING ON LIBRARY WITH IRREP Rank 1 AT COMPLEXITY 1 ---
--- WORKING ON LIBRARY WITH IRREP Antisymmetric rank 2 AT COMPLEXITY 1 ---
--- WORKING ON LIBRARY WITH IRREP Symmetric trace-free rank 2 AT COMPLEXITY 1 ---
--- WORKING ON LIBRARY WITH IRREP Rank 0 AT COMPLEXITY 2 ---
[0.01 s]
Identified model: ∂α u_α = 0 (order 2, residual 1.66e-13)
(Accuracy = 1.00e+00)
--- WORKING ON LIBRARY WITH IRREP Rank 1 AT COMPLEXITY 2 ---
--- WORKING ON LIBRARY WITH IRREP Antisymmetric rank 2 AT COMPLEXITY 2 ---
--- WORKING ON LIBRARY WITH IRREP Symmetric trace-free rank 2 AT COMPLEXITY 2 ---
--- WORKING ON LIBRARY WITH IRREP Rank 0 AT COMPLEXITY 3 ---
--- WORKING ON LIBRARY WITH IRREP Rank 1 AT COMPLEXITY 3 ---
[0.01 s]
Identified model: ∂α p + 1 · u_β · ∂β u_α + -0.0001 · ∂β² u_α + 1 · ∂t u_α = 0 (order 3, residual 1.16e-07)
(Accuracy = 7.37e-08)
--- WORKING ON LIBRARY WITH IRREP Antisymmetric rank 2 AT COMPLEXITY 3 ---
--- WORKING ON LI

In [9]:
Q = libs[srd.irreps[-1]].Q
terms = libs[srd.irreps[-1]].terms
opts['scaler'].reset_inds(list(range(len(terms))))

IOI = [9, 26, 23, 35]
print(terms[-14])
print(len(terms)-14)
print(terms[22])
print([terms[i] for i in IOI])

u_γ · ∂α ∂β u_γ
22
u_γ · ∂α ∂β u_γ
[∂α p · ∂t u_β, ∂α u_β, u_γ · ∂α ∂γ u_β, ∂t² ∂α u_β]


In [10]:
# coeffs = np.zeros(len(terms))
# coeffs[9] = 1
# coeffs[23] = -1e-4
# coeffs[26] = 1
# coeffs[35] = 1
# opts['scaler'].reset_inds([9, 23, 26, 35])
# reg_result = sparse_reg_bf(Q, **opts)
# #lambd = evaluate_model(Q, coeffs, opts['scaler'], opts['residual'])
# print(reg_result.xi[np.ix_([9, 23, 26, 35])], reg_result.lambd)
# #print(np.linalg.norm(Q[:, 33])/libs[irrep].col_weights[33])
# #print(np.linalg.norm(Q, axis=0)/libs[irrep].col_weights)

In [11]:
i=22
opts['inhomog'] = True
opts['inhomog_col'] = i
opts['verbose'] = False
opts['term_names'] = terms
print(terms[i])
reg_result = sparse_reg_bf(Q, **opts)
print(reg_result.xi, reg_result.lambd)
for i, x in enumerate(reg_result.xi):
    if x!=0:
        print('term', i, '--', reg_result.xi[i], '*', terms[i])

u_γ · ∂α ∂β u_γ
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. -1.  1. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] 5.229310941928285e-16
term 20 -- -0.999999999833966 * u_α · ∂γ² u_β
term 21 -- 1.0 * u_α · ∂β ∂γ u_γ
term 22 -- -0.9999999998339651 * u_γ · ∂α ∂β u_γ
term 23 -- 0.9999999998339658 * u_γ · ∂α ∂γ u_β


In [12]:
libs[srd.irreps[1]].terms

[p · p · ∂α p,
 p · p · u_α,
 p · p · ∂t u_α,
 p · ∂α p,
 p · ∂t p · u_α,
 p · ∂t ∂α p,
 p · u_α,
 p · u_β · ∂α u_β,
 p · u_α · ∂β u_β,
 p · u_β · ∂β u_α,
 p · ∂α ∂β u_β,
 p · ∂β² u_α,
 p · ∂t u_α,
 p · ∂t² u_α,
 ∂α p,
 ∂α p · ∂t p,
 ∂β p · u_α · u_β,
 ∂α p · u_β · u_β,
 ∂β p · ∂β u_α,
 ∂β p · ∂α u_β,
 ∂α p · ∂β u_β,
 ∂β² p · u_α,
 ∂α ∂β p · u_β,
 ∂α ∂β² p,
 ∂t p · u_α,
 ∂t p · ∂t u_α,
 ∂t ∂α p,
 ∂t² p · u_α,
 ∂t² ∂α p,
 u_α,
 u_α · u_β · u_β,
 u_β · u_β · ∂t u_α,
 u_α · u_β · ∂t u_β,
 u_β · ∂α u_β,
 u_α · ∂β u_β,
 u_β · ∂β u_α,
 u_α · ∂t ∂β u_β,
 u_β · ∂t ∂β u_α,
 u_β · ∂t ∂α u_β,
 ∂β u_β · ∂t u_α,
 ∂α u_β · ∂t u_β,
 ∂β u_α · ∂t u_β,
 ∂α ∂β u_β,
 ∂β² u_α,
 ∂t u_α,
 ∂t ∂α ∂β u_β,
 ∂t ∂β² u_α,
 ∂t² u_α,
 ∂t³ u_α]

In [13]:
print(srd.scale_dict)
for irrep in srd.irreps:
    # don't forget preprocessing
    Q = srd.libs[irrep].Q/srd.libs[irrep].col_weights # reweight columns
    for i in range(Q.shape[0]): # normalize rows
        Q[i, :] /= np.linalg.norm(Q[i, :])
    [U, S, V] = np.linalg.svd(Q)
    print(np.linalg.norm(Q)/max(S))

{'p': {'mean': np.float64(0.08393930372103373), 'std': np.float64(0.08393930372103378)}, 'u': {'mean': np.float64(0.28155178895306154), 'std': np.float64(0.28155178895306104)}}
1.4423652172180663
1.6032009894780117
1.1539319062472333
1.2954887186280073


In [14]:
print([reg_result.sublibrary for reg_result in reg_results])

[[p, p · p, ∂t p, u_α · u_α, ∂α u_α], [p · p · u_α, p · ∂α p, p · u_α, p · ∂t u_α, ∂α p, ∂t p · u_α, ∂t ∂α p, u_α, u_α · u_β · u_β, u_β · ∂α u_β, u_β · ∂β u_α, ∂β² u_α, ∂t u_α, ∂t² u_α], [p, p · p, p · p · p, p · p · ∂t p, p · ∂α p · u_α, p · ∂α² p, p · ∂t p, p · ∂t² p, p · u_α · u_α, p · u_α · ∂t u_α, ∂α p · ∂α p, ∂α p · u_α, ∂α p · ∂t u_α, ∂α² p, ∂t p, ∂t p · ∂t p, ∂t p · u_α · u_α, ∂t ∂α p · u_α, ∂t ∂α² p, ∂t² p, ∂t³ p, u_α · u_α, u_α · u_β · ∂α u_β, u_α · ∂β² u_α, u_α · ∂t u_α, u_α · ∂t² u_α, ∂α u_β · ∂α u_β, ∂α u_β · ∂β u_α, ∂t u_α · ∂t u_α], [p, p · p, p · p · p, p · p · ∂t p, p · ∂α p · u_α, p · ∂α² p, p · ∂t p, p · ∂t² p, p · u_α · u_α, p · u_α · ∂t u_α, ∂α p · ∂α p, ∂α p · u_α, ∂α p · ∂t u_α, ∂α² p, ∂t p, ∂t p · ∂t p, ∂t p · u_α · u_α, ∂t ∂α p · u_α, ∂t ∂α² p, ∂t² p, ∂t³ p, u_α · u_α, u_α · u_β · ∂α u_β, u_α · ∂β² u_α, u_α · ∂t u_α, u_α · ∂t² u_α, ∂α u_β · ∂α u_β, ∂t u_α · ∂t u_α], [p · p · ∂α u_β, p · ∂α p · u_β, p · u_α · ∂t u_β, p · ∂α u_β, p · ∂t ∂α u_β, ∂α p · u_β, ∂α p ·

In [15]:
from library import latexify

for irrep in srd.irreps:
    print(f"IRREP: {irrep}")
    print(latexify(str(srd.libs[irrep].terms)))
    #for term in srd.libs[irrep].terms:
    #    print(latexify(str(term))+",")

IRREP: Rank 0
[p, p \cdot p, p \cdot p \cdot p, p \cdot p \cdot \partial_t p, p \cdot p \cdot \partial_\alpha u_{\alpha}, p \cdot \partial_\alpha p \cdot u_{\alpha}, p \cdot \partial_\alpha^2 p, p \cdot \partial_t p, p \cdot \partial_t^2 p, p \cdot u_{\alpha} \cdot u_{\alpha}, p \cdot u_{\alpha} \cdot \partial_t u_{\alpha}, p \cdot \partial_\alpha u_{\alpha}, p \cdot \partial_t \partial_\alpha u_{\alpha}, \partial_\alpha p \cdot \partial_\alpha p, \partial_\alpha p \cdot u_{\alpha}, \partial_\alpha p \cdot \partial_t u_{\alpha}, \partial_\alpha^2 p, \partial_t p, \partial_t p \cdot \partial_t p, \partial_t p \cdot u_{\alpha} \cdot u_{\alpha}, \partial_t p \cdot \partial_\alpha u_{\alpha}, \partial_t \partial_\alpha p \cdot u_{\alpha}, \partial_t \partial_\alpha^2 p, \partial_t^2 p, \partial_t^3 p, u_{\alpha} \cdot u_{\alpha}, u_{\alpha} \cdot u_{\alpha} \cdot \partial_\beta u_{\beta}, u_{\alpha} \cdot u_{\beta} \cdot \partial_\beta u_{\alpha}, u_{\alpha} \cdot u_{\beta} \cdot \partial_

In [16]:
U, S, V = np.linalg.svd(Q)
print(S/S[0])
print("First", V[:, -1])
print("Second", V[:, -2])
print("Third", V[:, -3])
print("Fourth", V[:, -4])

[1.000e+00 4.778e-01 3.369e-01 3.116e-01 2.397e-01 2.200e-01 2.082e-01
 1.411e-01 1.352e-01 1.205e-01 1.177e-01 8.925e-02 7.250e-02 5.158e-02
 4.842e-02 4.286e-02 3.503e-02 2.947e-02 2.128e-02 1.841e-02 1.765e-02
 1.289e-02 1.045e-02 8.198e-03 6.667e-03 5.084e-03 2.870e-06 3.682e-07
 1.789e-07 1.061e-07 4.146e-08 1.006e-09 7.840e-17 7.840e-17 7.840e-17
 7.840e-17]
First [ 2.979e-01 -2.849e-02  2.096e-01  3.964e-01 -5.145e-01 -5.461e-01
  2.815e-01 -1.374e-01 -2.008e-02  2.777e-02  1.957e-02  8.555e-02
  1.509e-02 -5.522e-03  1.477e-01  7.827e-02 -9.563e-02  3.310e-02
  8.855e-04  8.202e-03 -9.757e-04  7.636e-03 -7.726e-03  2.707e-02
  6.309e-03  2.831e-03  3.623e-06 -5.629e-07 -8.081e-08 -1.741e-08
 -1.898e-08 -3.689e-10 -8.497e-17  7.092e-18 -1.358e-18 -2.399e-16]
Second [-3.854e-04  1.309e-01  1.863e-01  2.879e-01  1.005e-01  2.171e-01
 -7.070e-02  1.371e-03 -5.119e-02  2.021e-01  2.101e-01 -3.782e-02
  9.287e-02  6.358e-02 -4.137e-02  2.590e-01 -2.433e-01 -1.063e-01
 -1.404e-01  1.6